# Text generation with RNN
* [source site](https://www.tensorflow.org/beta/tutorials/text/text_generation)
* [see also Karpathy's blog](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [6]:
sorted(set('First Citizen:First, you know Caius Marcius is chief enemy to the people.'))

[' ',
 ',',
 '.',
 ':',
 'C',
 'F',
 'M',
 'a',
 'c',
 'e',
 'f',
 'h',
 'i',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'r',
 's',
 't',
 'u',
 'w',
 'y',
 'z']

In [7]:
char2idx = {u:i for i, u in enumerate(vocab)}
char2idx

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36,
 'Y': 37,
 'Z': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

In [8]:
idx2char = np.array(vocab)
idx2char

array(['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
       'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'],
      dtype='<U1')

In [9]:
text_as_int = np.array([char2idx[c] for c in text])

In [10]:
text_as_int[:250]

array([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43,
       44, 53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39,
       52, 63,  1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1,
       51, 43,  1, 57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31,
       54, 43, 39, 49,  6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56,
       57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39,
       56, 43,  1, 39, 50, 50,  1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56,
       39, 58, 46, 43, 56,  1, 58, 53,  1, 42, 47, 43,  1, 58, 46, 39, 52,
        1, 58, 53,  1, 44, 39, 51, 47, 57, 46, 12,  0,  0, 13, 50, 50, 10,
        0, 30, 43, 57, 53, 50, 60, 43, 42,  8,  1, 56, 43, 57, 53, 50, 60,
       43, 42,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43,
       52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63, 53, 59,  1, 49, 52, 53,
       61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41, 47, 59, 57,  1, 47,
       57,  1, 41, 46, 47

In [11]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}') # actually I already see the inside of char2idx dictionary, so it doesn't to be necessary 

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [12]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


### Above codes
1. text mining
2. sorted the character and made dictionary to match each character to int.

In [13]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//seq_length

In [14]:
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)


In [15]:
for i in char_dataset.take(5):
    print(idx2char[i.numpy()]) # i is int, thus the code work without .numpy()

F
i
r
s
t


In [16]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()]))) # each batch has 101 character, here 505 character will output.

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [17]:
len('First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi')

505

In [18]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [19]:
dataset.take(1)

<TakeDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

In [20]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()]))) # the reason to make the size batch as 100+1.
                                                                         # the size of input and target will be 100.

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [21]:
len('First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou')

100

In [22]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [23]:
input_example

<tf.Tensor: id=65, shape=(100,), dtype=int64, numpy=
array([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43,
       44, 53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39,
       52, 63,  1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1,
       51, 43,  1, 57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31,
       54, 43, 39, 49,  6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56,
       57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 37, 53, 59])>

## Create training batches

In [24]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [25]:
# Length of the vocabulary in chars
vocab_size = len(vocab) # 65.

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [26]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
        ])
    return model
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [29]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy() # converting multi dimentional array into 1D array.

In [30]:
sampled_indices

array([ 0, 64, 54, 18, 36, 11, 20, 50, 45, 53, 44, 46, 41,  3, 14, 64, 47,
       11, 47, 45, 58, 26, 53, 11, 49, 13, 15,  1,  7, 15, 22,  9, 52, 13,
       56, 55,  7,  1,  2, 39, 50, 34,  0, 45, 22, 27, 52, 12, 42, 26, 21,
        2, 34,  2, 10, 62, 17, 54, 23, 44, 12, 53, 38, 48, 40, 37, 36, 61,
       58, 10, 58,  6, 16, 57, 26, 53, 55, 21,  1, 42, 34, 25, 23, 35,  1,
       29, 39, 44, 50,  8, 59, 63, 34, 29, 31, 12,  2,  2,  2, 12])

In [31]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ]))) # without training.

Input: 
 'love moderately; long love doth so;\nToo swift arrives as tardy as too slow.\nHere comes the lady: O, '

Next Char Predictions: 
 '\nzpFX;Hlgofhc$Bzi;igtNo;kAC -CJ3nArq- !alV\ngJOn?dNI!V!:xEpKf?oZjbYXwt:t,DsNoqI dVMKW Qafl.uyVQS?!!!?'


In [32]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1754885


In [33]:
model.compile(optimizer='adam', loss=loss)

In [34]:
# Directory where the checkpoints will be saved
checkpoint_dir = './for_generating_RNN'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [35]:
EPOCHS=30
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
172/172 [==============================] - 11s 63ms/step - loss: 2.5564
Epoch 2/30
172/172 [==============================] - 10s 60ms/step - loss: 1.8670
Epoch 3/30
172/172 [==============================] - 10s 59ms/step - loss: 1.6229
Epoch 4/30
172/172 [==============================] - 10s 59ms/step - loss: 1.4913
Epoch 5/30
172/172 [==============================] - 10s 59ms/step - loss: 1.4102
Epoch 6/30
172/172 [==============================] - 10s 59ms/step - loss: 1.3506
Epoch 7/30
172/172 [==============================] - 10s 57ms/step - loss: 1.3007
Epoch 8/30
172/172 [==============================] - 10s 57ms/step - loss: 1.2549
Epoch 9/30
172/172 [==============================] - 10s 57ms/step - loss: 1.2116
Epoch 10/30
172/172 [==============================] - 10s 57ms/step - loss: 1.1707
Epoch 11/30
172/172 [==============================] - 10s 58ms/step - loss: 1.1311
Epoch 12/30
172/172 [==============================] - 10s 58ms/step - loss: 1.0945
E

In [36]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [37]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy() # converting multi dimentional array into 1D array.

In [38]:
sampled_indices

array([53, 59, 43,  1, 39, 53, 59, 43, 56, 57, 58, 43, 50, 63,  1,  1, 21,
       53, 52, 45,  1, 50, 53, 60, 43,  7, 35, 47, 58, 46,  1, 57, 53,  8,
        0, 32, 46, 53,  1, 57, 55, 47, 44, 58,  1, 39, 56, 43, 47, 60, 43,
       57,  1, 39, 57,  1, 40, 53, 40, 42, 63,  1, 39, 57,  1,  5, 53,  1,
        1, 46, 50, 39, 61,  1,  1,  0, 43,  1, 43,  1, 41, 53, 51, 43, 57,
        1, 24, 46, 43,  1, 50, 39, 64, 47, 10,  1, 47,  6,  1, 58])

In [39]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ]))) # without training.

Input: 
 'love moderately; long love doth so;\nToo swift arrives as tardy as too slow.\nHere comes the lady: O, '

Next Char Predictions: 
 "oue aouerstely  Iong love-With so.\nTho sqift areives as bobdy as 'o  hlaw  \ne e comes Lhe lazi: i, t"


### make model's input as 1

In [40]:
tf.train.latest_checkpoint(checkpoint_dir)

'./for_generating_RNN/ckpt_30'

In [41]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [42]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
lstm_1 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [49]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [52]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: YYUCY!CIZEGETCAIQLWYOFATGYYZZTZIACENGCEGVICEYTCHE:


First Citizen:
It was a judge, let us to him and good.
Not I'll resign unto Lucentio.

TRANIO:
And is this man did to oce for your king?

HENRY PERCY:
Then pardon me, and so Angelo:
Indeed, stark you.

Both Citizen:
And so did so? Get that bears more sent
Un ill spirit's royalthy am I, sir.

SAMPSON:
Nor aimed on me?
I have said, look now comes at the leasing seeks,
They made you pardon him, 'tis nour as I,
IS at these world goes, and made a sim
The most mates; but ignores, and let them nesty--
Thou slewing in me to strike? Now, pious sir,
Whence come to kill my talk and to your gains.

KING RICHARD II:
This name me not, nor an under young;
But I'll swear this;
The cover of the white and as much comfort this fair colours, I
Say, 'gainst the other were they trust he sayings
To be past to it, and many a looking on's three is she stoop
To sunset the city that's to die, in time with pity
Than hand it garland in him.

MIRANDA:
Be a

In [45]:
input_eval = [char2idx[s] for s in 'ROMEO: ']
input_eval

[30, 27, 25, 17, 27, 10, 1]

In [46]:
input_eval = tf.expand_dims(input_eval, 0)
input_eval

<tf.Tensor: id=159809, shape=(1, 7), dtype=int32, numpy=array([[30, 27, 25, 17, 27, 10,  1]], dtype=int32)>

In [47]:
sample_result = model(input_eval)
sampled_result = tf.random.categorical(sample_result[0], num_samples=1)
sampled_result = tf.squeeze(sampled_result,axis=-1).numpy() # converting multi dimentional array into 1D array.

In [53]:
print(repr("".join(idx2char[sampled_result]))) 

'IRPN:\nI'


## advanced: customized training

In [54]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [55]:
optimizer = tf.keras.optimizers.Adam()

In [56]:
@tf.function
def train_step(inp, target):
    with tf.GradientTape() as tape:
        predictions = model(inp)
        loss = tf.reduce_mean(
              tf.keras.losses.sparse_categorical_crossentropy(
                target, predictions, from_logits=True))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

In [57]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  # initializing the hidden state at the start of every epoch
  # initally hidden is None
  hidden = model.reset_states()

  for (batch_n, (inp, target)) in enumerate(dataset):
    loss = train_step(inp, target)

    if batch_n % 100 == 0:
      template = 'Epoch {} Batch {} Loss {}'
      print(template.format(epoch+1, batch_n, loss))

  # saving (checkpoint) the model every 5 epochs
  if (epoch + 1) % 5 == 0:
    model.save_weights(checkpoint_prefix.format(epoch=epoch))

  print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
  print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

W0717 16:22:53.036100 140075295348544 util.py:244] Unresolved object in checkpoint: (root).optimizer
W0717 16:22:53.036699 140075295348544 util.py:244] Unresolved object in checkpoint: (root).optimizer.iter
W0717 16:22:53.037014 140075295348544 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_1
W0717 16:22:53.037289 140075295348544 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_2
W0717 16:22:53.037549 140075295348544 util.py:244] Unresolved object in checkpoint: (root).optimizer.decay
W0717 16:22:53.037808 140075295348544 util.py:244] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0717 16:22:53.038078 140075295348544 util.py:244] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-0.embeddings
W0717 16:22:53.038480 140075295348544 util.py:244] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-2.kernel
W0717 16:22:53.038883 140075295348544 util.py:2

Epoch 1 Batch 0 Loss 4.1739068031311035
Epoch 1 Batch 100 Loss 2.3141682147979736
Epoch 1 Loss 2.0871
Time taken for 1 epoch 10.562193870544434 sec

Epoch 2 Batch 0 Loss 2.0485711097717285
Epoch 2 Batch 100 Loss 1.860837697982788
Epoch 2 Loss 1.7528
Time taken for 1 epoch 8.912753105163574 sec

Epoch 3 Batch 0 Loss 1.7017529010772705
Epoch 3 Batch 100 Loss 1.650705099105835
Epoch 3 Loss 1.5746
Time taken for 1 epoch 8.8881094455719 sec

Epoch 4 Batch 0 Loss 1.5330157279968262
Epoch 4 Batch 100 Loss 1.5272574424743652
Epoch 4 Loss 1.4673
Time taken for 1 epoch 8.852967977523804 sec

Epoch 5 Batch 0 Loss 1.4346429109573364
Epoch 5 Batch 100 Loss 1.4540618658065796
Epoch 5 Loss 1.3960
Time taken for 1 epoch 8.983646392822266 sec

Epoch 6 Batch 0 Loss 1.371189832687378
Epoch 6 Batch 100 Loss 1.3981536626815796
Epoch 6 Loss 1.3407
Time taken for 1 epoch 8.802841901779175 sec

Epoch 7 Batch 0 Loss 1.319631814956665
Epoch 7 Batch 100 Loss 1.352746605873108
Epoch 7 Loss 1.2920
Time taken for 1

In [59]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_2 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [60]:
tf.train.latest_checkpoint(checkpoint_dir)

'./for_generating_RNN/ckpt_9'

In [61]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [62]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
lstm_3 (LSTM)                (1, None, 1024)           5246976   
_________________________________________________________________
dense_3 (Dense)              (1, None, 65)             66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [64]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: if thou arm'd forth!
As we have learn a sepulity of lime towers,
And by my father's most I but a dot um.
And my shose courtest loves mine evils
And philvess eyes and within. What cock bits my swift
Quean carce but in place? By me
End here for held my heart shame great move's dost.
Therefore yourself, he shall;
Be remedy; in this report of welcome gottenness,
When he shall supply them full tce fight
That the sea parleman as a himanement.

AUFIDIO:
I pray, my sey.

First Conspirator,
Dody to our bretshorn, frighteous present. An her most sort! I would you cave laughter long shall weep?

QUEE Mine but this even news to take what thou artsweer
Is true the way to cherity will rate
He saking and first follows.

GRUMIO:

ISABELLA:
And weapon he, for she nemesparticils?

POMPEY:
O; ladple dehite
Be pardon.

CORDOLANUS:
Sir: I can complain the king my due.

QUEEN MARGARET:
For sadve is le, filest thou, if uny makes a cleaven
On so ebjuct: by your laging in your beast,
She shall be spent 

## Summary

* RNN can receive varying size of input text.
* When you want to train RNN with one characeter, you have to input massive texts with sliding the texts with one character step.
* When you want to test the model, you might want to rebuild the model with one batch model. and reload the trained weight.